In [ ]:
# Cell 1: Header and Setup
import sys
sys.path.insert(0, '..')

from utils.notebook_utils import display_header, display_toc, check_dependency, conclusion_box
from utils.system_info import display_system_info
from utils.benchmark import Benchmark, BenchmarkResult, ComparisonTable
from utils.charts import setup_style, bar_comparison, throughput_comparison, COLORS

display_header('Model Registry Comparison', 'SynaDB vs Hugging Face Hub')

In [ ]:
# Cell 2: Table of Contents
sections = [
    ('Introduction', 'introduction'),
    ('Setup', 'setup'),
    ('Transformer Model Storage', 'transformer-storage'),
    ('Load Time Comparison', 'load-time'),
    ('Version Management', 'version-management'),
    ('Fine-tuned Models', 'fine-tuned'),
    ('Offline Usage', 'offline'),
    ('Results Summary', 'results'),
    ('Conclusions', 'conclusions'),
]
display_toc(sections)

## 📌 Introduction <a id="introduction"></a>

This notebook compares **SynaDB's ModelRegistry** against **Hugging Face Hub** for transformer model storage.

| System | Type | Key Features |
|--------|------|-------------|
| **SynaDB** | Local/Embedded | Single-file, offline-first, SHA-256 checksums |
| **Hugging Face Hub** | Cloud | Largest model repository, community sharing, model cards |

### What We'll Compare

- **Transformer model storage** - Saving and loading model weights
- **Load time** - Local SynaDB vs HF Hub download
- **Version management** - Local versioning vs Hub revisions
- **Fine-tuned models** - Storing custom model weights
- **Offline usage** - Air-gapped environment support

### Use Cases

- **SynaDB**: Local development, edge deployment, air-gapped environments
- **HF Hub**: Model sharing, community collaboration, cloud deployment

In [ ]:
# Cell 4: System Info
display_system_info()

## 🔧 Setup <a id="setup"></a>

Let's set up our test environment for model storage comparison.

In [ ]:
# Cell 6: Check Dependencies and Imports
import numpy as np
import time
import os
import shutil
import tempfile
import hashlib
from pathlib import Path
import matplotlib.pyplot as plt

# Check for SynaDB
HAS_SYNADB = check_dependency('synadb', 'pip install synadb')

# Check for Hugging Face Hub
HAS_HF_HUB = check_dependency('huggingface_hub', 'pip install huggingface_hub')

# Check for transformers (optional, for real model tests)
HAS_TRANSFORMERS = check_dependency('transformers', 'pip install transformers')

# Apply consistent styling
setup_style()

In [ ]:
# Cell 7: Configuration
# Test configuration
SEED = 42
NUM_VERSIONS = 5  # Number of model versions to test

# Simulated model sizes (in MB)
# Real transformer models: BERT-base ~440MB, GPT-2 ~500MB, etc.
SIMULATED_MODEL_SIZE_MB = 50  # Smaller for faster testing

print('Test Configuration:')
print(f'  Simulated model size: {SIMULATED_MODEL_SIZE_MB}MB')
print(f'  Number of versions: {NUM_VERSIONS}')

np.random.seed(SEED)

In [ ]:
# Cell 8: Create Temp Directory
temp_dir = tempfile.mkdtemp(prefix='synadb_hf_benchmark_')
print(f'Using temp directory: {temp_dir}')

# Paths
synadb_path = os.path.join(temp_dir, 'synadb_models.db')
hf_cache_path = os.path.join(temp_dir, 'hf_cache')

In [ ]:
# Cell 9: Generate Simulated Model Data
# Generate simulated transformer model weights
# In practice, these would be actual PyTorch state_dict or safetensors

def generate_model_weights(size_mb, version):
    """Generate simulated model weights with version-specific variations."""
    np.random.seed(SEED + version)
    return np.random.bytes(size_mb * 1024 * 1024)

# Generate multiple versions
model_versions = {}
for v in range(NUM_VERSIONS):
    model_versions[v] = generate_model_weights(SIMULATED_MODEL_SIZE_MB, v)
    print(f'✓ Generated model version {v+1} ({SIMULATED_MODEL_SIZE_MB}MB)')

# Model metadata (simulating HF model card info)
model_metadata = {
    'model_type': 'bert',
    'hidden_size': 768,
    'num_attention_heads': 12,
    'num_hidden_layers': 12,
    'vocab_size': 30522,
}

## 🤖 Transformer Model Storage <a id="transformer-storage"></a>

Comparing how each system stores transformer model weights.

In [ ]:
# Cell 11: SynaDB Transformer Storage
synadb_save_times = []
synadb_registry = None

if HAS_SYNADB:
    from synadb import ModelRegistry
    
    print('Storing transformer models in SynaDB...')
    synadb_registry = ModelRegistry(synadb_path)
    
    for v in range(NUM_VERSIONS):
        # Time model save
        start = time.perf_counter()
        version_info = synadb_registry.save_model(
            'bert-base-custom',
            model_versions[v],
            {**model_metadata, 'fine_tuned_epoch': str(v * 10)}
        )
        elapsed = (time.perf_counter() - start) * 1000
        synadb_save_times.append(elapsed)
        print(f'  v{v+1}: {elapsed:.2f}ms')
    
    throughput = SIMULATED_MODEL_SIZE_MB * NUM_VERSIONS * 1000 / sum(synadb_save_times)
    print(f'\n✓ Saved {NUM_VERSIONS} versions')
    print(f'  Mean: {np.mean(synadb_save_times):.2f}ms')
    print(f'  Throughput: {throughput:.1f} MB/s')
else:
    print('⚠️ SynaDB not available')

In [ ]:
# Cell 12: HF Hub Storage Pattern (Simulated)
hf_save_times = []

# Note: Actual HF Hub upload requires authentication and network
# We simulate the local file operations that HF Hub performs

print('Simulating Hugging Face Hub storage pattern...')
os.makedirs(hf_cache_path, exist_ok=True)

for v in range(NUM_VERSIONS):
    # Simulate HF Hub's local caching behavior
    start = time.perf_counter()
    
    # HF Hub stores models in a specific directory structure
    model_dir = os.path.join(hf_cache_path, f'models--bert-base-custom', f'snapshots', f'v{v}')
    os.makedirs(model_dir, exist_ok=True)
    
    # Write model weights (simulating safetensors or pytorch_model.bin)
    model_file = os.path.join(model_dir, 'pytorch_model.bin')
    with open(model_file, 'wb') as f:
        f.write(model_versions[v])
    
    # Write config (simulating config.json)
    import json
    config_file = os.path.join(model_dir, 'config.json')
    with open(config_file, 'w') as f:
        json.dump(model_metadata, f)
    
    elapsed = (time.perf_counter() - start) * 1000
    hf_save_times.append(elapsed)
    print(f'  v{v+1}: {elapsed:.2f}ms')

throughput = SIMULATED_MODEL_SIZE_MB * NUM_VERSIONS * 1000 / sum(hf_save_times)
print(f'\n✓ Saved {NUM_VERSIONS} versions (local cache simulation)')
print(f'  Mean: {np.mean(hf_save_times):.2f}ms')
print(f'  Throughput: {throughput:.1f} MB/s')

In [ ]:
# Cell 13: Storage Comparison Visualization
save_throughput = {}

if synadb_save_times:
    save_throughput['SynaDB'] = SIMULATED_MODEL_SIZE_MB * NUM_VERSIONS * 1000 / sum(synadb_save_times)

if hf_save_times:
    save_throughput['HF Hub (local)'] = SIMULATED_MODEL_SIZE_MB * NUM_VERSIONS * 1000 / sum(hf_save_times)

if save_throughput:
    fig = throughput_comparison(
        save_throughput,
        title=f'Model Storage Throughput ({SIMULATED_MODEL_SIZE_MB}MB models)',
        ylabel='MB/second'
    )
    plt.show()

## ⏱️ Load Time Comparison <a id="load-time"></a>

Comparing local load times vs network download times.

In [ ]:
# Cell 15: SynaDB Load Time
synadb_load_times = []

if HAS_SYNADB and synadb_registry:
    print('Benchmarking SynaDB model load...')
    
    for v in range(1, NUM_VERSIONS + 1):
        # Time model load with checksum verification
        start = time.perf_counter()
        data, info = synadb_registry.load_model('bert-base-custom', version=v)
        elapsed = (time.perf_counter() - start) * 1000
        synadb_load_times.append(elapsed)
        print(f'  v{v}: {elapsed:.2f}ms (verified)')
    
    throughput = SIMULATED_MODEL_SIZE_MB * NUM_VERSIONS * 1000 / sum(synadb_load_times)
    print(f'\n✓ Loaded {NUM_VERSIONS} versions')
    print(f'  Mean: {np.mean(synadb_load_times):.2f}ms')
    print(f'  Throughput: {throughput:.1f} MB/s')
else:
    print('⚠️ SynaDB not available')

In [ ]:
# Cell 16: HF Hub Load Time (Local Cache)
hf_load_times = []

print('Benchmarking HF Hub local cache load...')

for v in range(NUM_VERSIONS):
    model_file = os.path.join(hf_cache_path, f'models--bert-base-custom', f'snapshots', f'v{v}', 'pytorch_model.bin')
    
    if os.path.exists(model_file):
        start = time.perf_counter()
        with open(model_file, 'rb') as f:
            data = f.read()
        elapsed = (time.perf_counter() - start) * 1000
        hf_load_times.append(elapsed)
        print(f'  v{v+1}: {elapsed:.2f}ms')

if hf_load_times:
    throughput = SIMULATED_MODEL_SIZE_MB * len(hf_load_times) * 1000 / sum(hf_load_times)
    print(f'\n✓ Loaded {len(hf_load_times)} versions from cache')
    print(f'  Mean: {np.mean(hf_load_times):.2f}ms')
    print(f'  Throughput: {throughput:.1f} MB/s')

In [ ]:
# Cell 17: Load Time Comparison
from IPython.display import display, Markdown

# Estimated HF Hub download times (based on typical network speeds)
# Assuming 50Mbps connection for a 50MB model
estimated_download_time_ms = SIMULATED_MODEL_SIZE_MB * 8 / 50 * 1000  # ~8 seconds

# Format values safely (handle empty lists)
synadb_load_str = f'{np.mean(synadb_load_times):.0f}' if synadb_load_times else 'N/A'
hf_load_str = f'{np.mean(hf_load_times):.0f}' if hf_load_times else 'N/A'

load_comparison = f'''
### Load Time Comparison

| Scenario | SynaDB | HF Hub (cached) | HF Hub (download) |
|----------|--------|-----------------|-------------------|
| **First load** | {synadb_load_str}ms | {hf_load_str}ms | ~{estimated_download_time_ms:.0f}ms* |
| **Cached load** | {synadb_load_str}ms | {hf_load_str}ms | {hf_load_str}ms |
| **Checksum verification** | ✅ Automatic | ❌ None | ❌ None |

*Estimated for {SIMULATED_MODEL_SIZE_MB}MB model on 50Mbps connection

**Key Insight:** SynaDB provides consistent, fast load times with built-in integrity verification.
HF Hub requires network access for first load, then caches locally.
'''
display(Markdown(load_comparison))

## 📋 Version Management <a id="version-management"></a>

Comparing version management approaches.

In [ ]:
# Cell 19: SynaDB Version Management
if HAS_SYNADB and synadb_registry:
    print('SynaDB Version Management')
    print('=' * 50)
    
    # List versions
    versions = synadb_registry.list_versions('bert-base-custom')
    print(f'\nModel: bert-base-custom')
    print(f'Total versions: {len(versions)}')
    
    print('\nVersion History:')
    for v in versions:
        print(f'  v{v.version}: {v.stage}')
        print(f'    Checksum: {v.checksum[:24]}...')
else:
    print('⚠️ SynaDB not available')

In [ ]:
# Cell 20: Version Management Comparison
from IPython.display import display, Markdown

version_comparison = '''
### Version Management Comparison

| Feature | SynaDB | Hugging Face Hub |
|---------|--------|------------------|
| **Version identifier** | Sequential numbers | Git commit hashes |
| **Version listing** | `list_versions()` | `list_repo_refs()` |
| **Load specific version** | `load_model(name, version=N)` | `revision="commit_hash"` |
| **Metadata per version** | ✅ Built-in | ✅ Model cards |
| **Offline versioning** | ✅ Full support | ❌ Requires network |
| **Private versions** | ✅ Local only | ✅ Private repos |

**SynaDB Approach:**
```python
# Simple sequential versioning
registry.save_model("my-model", weights, metadata)  # Auto v1, v2, v3...
data, info = registry.load_model("my-model", version=2)
```

**HF Hub Approach:**
```python
# Git-based versioning
model = AutoModel.from_pretrained("user/model", revision="abc123")
```
'''
display(Markdown(version_comparison))

## 🎯 Fine-tuned Models <a id="fine-tuned"></a>

Demonstrating storage of fine-tuned model weights.

In [ ]:
# Cell 22: Fine-tuned Model Storage Demo
if HAS_SYNADB and synadb_registry:
    print('Storing Fine-tuned Models in SynaDB')
    print('=' * 50)
    
    # Simulate fine-tuning iterations
    fine_tune_metadata = [
        {'task': 'sentiment', 'dataset': 'imdb', 'accuracy': '0.92', 'epochs': '3'},
        {'task': 'sentiment', 'dataset': 'imdb', 'accuracy': '0.94', 'epochs': '5'},
        {'task': 'sentiment', 'dataset': 'imdb', 'accuracy': '0.95', 'epochs': '10'},
    ]
    
    print('\nSaving fine-tuned checkpoints...')
    for i, meta in enumerate(fine_tune_metadata):
        # Generate slightly different weights for each checkpoint
        weights = generate_model_weights(10, 100 + i)  # 10MB for faster demo
        
        version = synadb_registry.save_model(
            'bert-sentiment-finetuned',
            weights,
            meta
        )
        print(f'  Checkpoint {i+1}: accuracy={meta["accuracy"]}, epochs={meta["epochs"]}')
    
    # Promote best model to production
    print('\n\nPromoting best model (v3) to Production...')
    synadb_registry.set_stage('bert-sentiment-finetuned', 3, 'Production')
    
    # Get production model
    prod = synadb_registry.get_production('bert-sentiment-finetuned')
    if prod:
        print(f'  ✓ Production model: v{prod.version}')
        print(f'  ✓ Checksum: {prod.checksum[:24]}...')
else:
    print('⚠️ SynaDB not available')

In [ ]:
# Cell 23: Fine-tuned Model Comparison
from IPython.display import display, Markdown

finetune_comparison = '''
### Fine-tuned Model Storage Comparison

| Feature | SynaDB | Hugging Face Hub |
|---------|--------|------------------|
| **Store checkpoints** | ✅ `save_model()` | ✅ `push_to_hub()` |
| **Track training metadata** | ✅ Built-in | ✅ Model cards |
| **Compare versions** | ✅ `list_versions()` | ✅ Hub UI |
| **Promote to production** | ✅ `set_stage()` | ⚠️ Manual tags |
| **Offline fine-tuning** | ✅ Full support | ❌ Requires network to push |
| **Private storage** | ✅ Local only | ✅ Private repos (paid) |

**Best Practice:** Use SynaDB for local development and checkpointing,
then push final models to HF Hub for sharing.
'''
display(Markdown(finetune_comparison))

## 🔌 Offline Usage <a id="offline"></a>

Demonstrating SynaDB's advantage in air-gapped environments.

In [ ]:
# Cell 25: Offline Usage Demonstration
from IPython.display import display, Markdown

offline_demo = '''
### Offline Usage Comparison

| Scenario | SynaDB | Hugging Face Hub |
|----------|--------|------------------|
| **Air-gapped environment** | ✅ Full functionality | ❌ No access |
| **Edge deployment** | ✅ Single file | ⚠️ Pre-download required |
| **No internet** | ✅ Works perfectly | ❌ Cache only |
| **Secure environments** | ✅ No network calls | ⚠️ Firewall issues |
| **Reproducibility** | ✅ Checksums | ⚠️ Network-dependent |

### SynaDB Offline Workflow

```python
# Works anywhere - no network required
from synadb import ModelRegistry

# Single file contains all models and versions
registry = ModelRegistry("models.db")

# Save model
registry.save_model("my-model", weights, metadata)

# Load with integrity verification
data, info = registry.load_model("my-model")
# Checksum automatically verified!
```

### HF Hub Offline Workflow

```python
# Requires pre-downloading models
from transformers import AutoModel

# First: Download while online
model = AutoModel.from_pretrained("bert-base-uncased")
model.save_pretrained("./local_model")

# Later: Load from local cache
model = AutoModel.from_pretrained("./local_model", local_files_only=True)
```
'''
display(Markdown(offline_demo))

In [ ]:
# Cell 26: Storage Size Comparison
def get_dir_size(path):
    """Get total size of a directory in bytes."""
    total = 0
    if os.path.isfile(path):
        return os.path.getsize(path)
    if not os.path.exists(path):
        return 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total += os.path.getsize(fp)
    return total

def count_files(path):
    """Count files in a directory."""
    if os.path.isfile(path):
        return 1
    if not os.path.exists(path):
        return 0
    count = 0
    for dirpath, dirnames, filenames in os.walk(path):
        count += len(filenames)
    return count

print('Storage Comparison')
print('=' * 60)

# SynaDB
if os.path.exists(synadb_path):
    size = get_dir_size(synadb_path)
    files = count_files(synadb_path)
    print(f'SynaDB: {size / (1024 * 1024):.2f} MB ({files} file)')

# HF Cache
if os.path.exists(hf_cache_path):
    size = get_dir_size(hf_cache_path)
    files = count_files(hf_cache_path)
    print(f'HF Cache: {size / (1024 * 1024):.2f} MB ({files} files)')

print('\n' + '=' * 60)
print('\nNote: SynaDB stores everything in a single portable file.')
print('HF Hub uses a complex directory structure with many files.')

## 📊 Results Summary <a id="results"></a>

Summarizing the comparison between SynaDB and Hugging Face Hub.

In [ ]:
# Cell 28: Results Summary
from IPython.display import display, Markdown

summary = '''
### Feature Comparison Summary

| Feature | SynaDB | Hugging Face Hub |
|---------|--------|------------------|
| **Storage type** | Single file | Cloud + local cache |
| **Offline support** | ✅ Full | ⚠️ Cache only |
| **Integrity verification** | ✅ SHA-256 | ❌ None |
| **Version management** | ✅ Sequential | ✅ Git-based |
| **Stage management** | ✅ Built-in | ⚠️ Manual tags |
| **Community sharing** | ❌ Local only | ✅ Excellent |
| **Model discovery** | ❌ None | ✅ Hub search |
| **Pre-trained models** | ❌ None | ✅ Thousands |
| **Setup complexity** | ✅ Zero config | ⚠️ Auth required |
| **Cost** | ✅ Free | ⚠️ Paid for private |

### When to Use Each

**Use SynaDB when:**
- Developing locally without network
- Deploying to edge devices
- Working in air-gapped environments
- Need integrity verification
- Want simple, portable storage

**Use HF Hub when:**
- Sharing models with the community
- Using pre-trained models
- Collaborating with teams
- Need model cards and documentation
- Want cloud-based storage
'''
display(Markdown(summary))

In [ ]:
# Cell 29: Performance Summary Chart
# Create comparison visualization
categories = ['Offline', 'Integrity', 'Simplicity', 'Sharing', 'Discovery']
synadb_scores = [1.0, 1.0, 1.0, 0.0, 0.0]
hf_scores = [0.3, 0.0, 0.5, 1.0, 1.0]

x = np.arange(len(categories))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, synadb_scores, width, label='SynaDB', color=COLORS['synadb'])
bars2 = ax.bar(x + width/2, hf_scores, width, label='HF Hub', color=COLORS['competitor'])

ax.set_ylabel('Score')
ax.set_title('SynaDB vs Hugging Face Hub - Feature Comparison')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.set_ylim(0, 1.2)
ax.legend()
ax.set_yticks([0, 0.5, 1])
ax.set_yticklabels(['None', 'Partial', 'Full'])

plt.tight_layout()
plt.show()

## 🎯 Conclusions <a id="conclusions"></a>

In [ ]:
# Cell 31: Conclusions
conclusion_box(
    title='Key Takeaways',
    points=[
        'SynaDB excels for local, offline, and edge deployment scenarios',
        'SHA-256 checksums provide strong integrity guarantees',
        'Single-file storage simplifies deployment and backup',
        'HF Hub is unmatched for community sharing and pre-trained models',
        'Both tools can complement each other in a workflow',
        'Use SynaDB for development, HF Hub for distribution'
    ],
    summary='SynaDB and HF Hub serve different needs - use both for a complete workflow.'
)

In [ ]:
# Cell 32: Cleanup
import shutil
try:
    shutil.rmtree(temp_dir)
    print(f'✓ Cleaned up temp directory: {temp_dir}')
except Exception as e:
    print(f'⚠️ Could not clean up: {e}')

---

**Next Steps:**
- Explore [LLM Framework integrations](../llm_frameworks/) for RAG applications
- Check out [End-to-End Pipeline](../specialized/18_end_to_end_pipeline.ipynb) for complete ML workflows
- See [GPU Performance](../specialized/15_gpu_performance.ipynb) for high-performance training